In [6]:
import redis
import time

class RedisQueue:
	def __init__(
		self,
		host: str = "localhost",
		port: int = 6379,
		db: int = 0
	):
		self.r = redis.Redis(host=host, port=port, db=db)
		self.p = self.r.pubsub()
		self.p.subscribe("c")
	
	def push(self, data: str):
		"""
		push data to list head in Redis

		publish a message
		"""
		self.r.lpush("l", data)
		return self.r.publish("c", data)
	
	def pop(self):
		"""
		loop waiting for a message from Redis.

		pop data from list tail in Redis
		"""
		while True:
			message = self.p.get_message()
			if message:
				break
			time.sleep(0.001)
		return self.r.rpop("l")

class Client:
	queue = RedisQueue()

	def consumer(self):
		"""
		loop receiving messages with queue.pop()

		every consumer ends upon receiving an "end" message

		and prints the number of messages received
		"""

		count = 0
		while True:
			message = self.queue.pop()
			count += 1
			if message == "end":
				return count
	
	def producer(self):
		"""
		loop sending messages with queue.push()

		every producer sends 10000 messages
		"""

		for i in range (1000):
			self.queue.push(str(i))

	def terminator(self):
		"""
		a parameter in the (main) controls

		if it should send "end" messages in the end

		ex: python producer.py -e 4
		"""
		self.queue.push("end")

# Client().consumer()
# Client().producer()
# Client().terminator()

KeyboardInterrupt: 